In [1]:
import datatable as dt

# Variables that contains the file location
from files import *
from functions import *

In [2]:
# if we modify the file we need to reload it with this
import importlib
import functions  #import the module here, so that it can be reloaded.

importlib.reload(functions)

<module 'functions' from '/DATA/myprograms_Python/MMSR/project/functions.py'>

In [3]:
DATA_TYPE = np.float16

# Data
Load the Data

In [4]:
genres = dt.fread(file_genres_2).to_pandas()
genres.set_index('id', inplace=True)

In [5]:
# Our special child
blf_logfluc = dt.fread(file_blf_logfluc)
blf_logfluc[dt.float64] = dt.float32
new_cols = ['id']
new_cols.extend(list(blf_logfluc.names[2:]))
new_cols = tuple(new_cols)
del blf_logfluc[:, -1]
blf_logfluc.names = new_cols
blf_logfluc = blf_logfluc.to_pandas()
blf_logfluc.set_index('id', inplace=True)

In [6]:
files = [
    # Lyrics
    file_tfidf_2,
    file_word2vec_2,
    file_bert_2,
    # Audio
    # file_essentia, # zeros
    file_blf_correlation,
    file_blf_deltaspectral,
    file_blf_spectral,
    file_blf_spectralcontrast,
    file_blf_vardeltaspectral,
    # file_blf_logfluc, # zeros
    blf_logfluc,
    file_mfcc_bow,
    file_mfcc_stats,
    # Video
    file_incp,
    file_resnet,
    file_vgg19,
]

In [7]:
functions = [get_cosine_similarity]

In [8]:
common_index = blf_logfluc.index.values

In [9]:
import csv


def load_data(file):
    if type(file) == str:
        with open(file, "r") as f:
            column_names = next(csv.reader(f, delimiter="\t"))
        dtypes = {
            x: np.float32
            for x in column_names
            if x != "id"
        }
        file = pd.read_csv(file, dtype=dtypes, index_col=0, delimiter="\t")
    file = file.loc[common_index]
    assert np.mean(file.index.values == blf_logfluc.index.values) == 1.0
    return file

In [10]:
# Extract readable names
names = []
for file in files:
    if type(file) == str:
        names.append(file.replace("./../task2/id_", "").replace(".tsv", ""))
    else:
        names.append("blf_logfluc")

# Late Fusion

In [11]:
def compute_in_batches_topIds(results: np.array, idx_values: np.array, top: int = 100, batches: int = 1):
    splits_b = np.array_split(results, batches, axis=0)
    return np.concatenate([idx_values[np.argsort(b * -1, axis=1)][:, :top] for b in tqdm(splits_b)], axis=0)

# Find optimal feature similarity weights

In [12]:
subset = np.random.choice(len(blf_logfluc), 1024 * 4, replace=False)
subset.shape

(4096,)

In [13]:
subset_ids = blf_logfluc.index.values[subset]

similarities = []

# for every data
i = 0
for d in tqdm(files, desc="Processing"):
    file = load_data(d).to_numpy()[subset, :].astype(np.float32)
    for f in functions:
        result = np.nan_to_num(f(file, file), copy=False)
        similarities.append(result)
    i += 1

print(f"{len(similarities)} similarity matrices")

Processing: 100%|██████████| 14/14 [02:57<00:00, 12.71s/it]

14 similarity matrices


In [14]:
def evaluate(similarities, weights):
    if len(similarities) == 1:
        accumulated_similarity = similarities[0]
    else:
        accumulated_similarity = np.zeros((len(subset), len(subset)))
        for sim, weight in zip(similarities, weights):
            accumulated_similarity += (sim * weight)

    top_ids = pd.DataFrame(compute_in_batches_topIds(accumulated_similarity, subset_ids, 100, 10), subset_ids)

    return {
        "MAP@10": meanAveragePrecision(top_ids, 10, genres),
        "MRR@10": meanReciprocalRank(top_ids, 10, genres),
        "NDCG@10": ndcgMean(top_ids, 10, genres)[1],
    }

In [15]:
print("Baseline", evaluate(similarities, np.zeros((len(similarities),))))

  0%|          | 0/4096 [00:00<?, ?it/s]/DATA/myprograms_Python/MMSR/project/functions.py:111: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:111: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:111: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:111: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:111: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:111: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_P

Baseline {'MAP@10': 0.5228031456203177, 'MRR@10': 0.6751056973896329, 'NDCG@10': nan}


In [16]:
print("Fair", evaluate(similarities, np.ones((len(similarities),))))

100%|██████████| 4096/4096 [00:02<00:00, 1810.08it/s]

Fair {'MAP@10': 0.8581923691107302, 'MRR@10': 1.0, 'NDCG@10': 0.9060482474110523}


In [17]:
weights = np.zeros((len(similarities),))
for i, sim in enumerate(similarities):
    result = np.nan_to_num(np.asarray(list(evaluate([sim], [1]).values())), copy=False)
    weights[i] = np.mean(result, where=result > 0)
print(weights)

 44%|████▎     | 1790/4096 [00:01<00:01, 1730.89it/s]/DATA/myprograms_Python/MMSR/project/functions.py:111: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
100%|██████████| 4096/4096 [00:02<00:00, 1775.62it/s]

[0.89834306 0.8990285  0.90240972 0.90317382 0.90478601 0.91075214
 0.9158896  0.90650437 0.91416988 0.90965728 0.90672442 0.90500545
 0.90552266 0.90272872]


In [18]:
print("Performance", evaluate(similarities, weights))

100%|██████████| 4096/4096 [00:02<00:00, 1809.65it/s]

Performance {'MAP@10': 0.858436658363794, 'MRR@10': 1.0, 'NDCG@10': 0.9062485966343592}


In [19]:
norm_weights = (weights - weights.min()) / (weights.max() - weights.min())
print(norm_weights)
print("Normalized Performance", evaluate(similarities, norm_weights))

[0.         0.03906398 0.23176407 0.27531116 0.36719205 0.70720943
 1.         0.4651235  0.90199083 0.64481174 0.47766435 0.37969793
 0.40917449 0.24994444]


100%|██████████| 4096/4096 [00:02<00:00, 1796.48it/s]

Normalized Performance {'MAP@10': 0.8599203391254143, 'MRR@10': 1.0, 'NDCG@10': 0.9064573547900158}


In [20]:
del similarities

# Final Processing
Normalized Performance seems to be the best approach

In [21]:
final_ids = blf_logfluc.index.values
final_accumulated_similarity = np.zeros((len(final_ids), len(final_ids)), dtype=DATA_TYPE)

In [22]:
# for every data
batches = 100
i = 0
for d in tqdm(files, desc="Processing"):
    file = load_data(d).to_numpy()
    for f in functions:
        splits = np.array_split(file, batches, axis=0)
        y = 0
        for b in tqdm(splits):
            result = f(file, b).astype(np.float32)

            result = np.nan_to_num(result, copy=False)

            # normalize
            np.subtract(result, result.mean(), out=result)

            # normalize std and apply weight
            result = np.multiply(result, norm_weights[i] / result[::64, ::64].std(), out=result)

            final_accumulated_similarity[:, y:y + b.shape[0]] += result
            y += b.shape[0]
        print("Sanity Check", final_accumulated_similarity[:1024, :1024].mean())
        i += 1

Processing:   7%|▋         | 1/14 [01:23<18:00, 83.09s/it]

Sanity Check 0.0



Processing:  14%|█▍        | 2/14 [02:24<14:04, 70.36s/it]

Sanity Check 0.000456



Processing:  21%|██▏       | 3/14 [03:40<13:21, 72.83s/it]

...

Sanity Check -0.1012



Processing:  93%|█████████▎| 13/14 [32:29<04:42, 282.38s/it]

Sanity Check -0.1123



Processing: 100%|██████████| 14/14 [46:35<00:00, 199.67s/it]

Sanity Check -0.121


In [23]:
# replace diagonals by -inf to prevent them being picked
np.fill_diagonal(final_accumulated_similarity, -np.inf)

In [24]:
np.save("final_similarity.p", final_accumulated_similarity)

In [26]:
top_ids = pd.DataFrame(compute_in_batches_topIds(final_accumulated_similarity, final_ids, 100, 100), final_ids)

100%|██████████| 100/100 [07:59<00:00,  4.79s/it]


In [27]:
dt.Frame(pd.DataFrame(top_ids, index=final_ids).reset_index()).to_csv('./top_ids_late_fusion.csv')

In [29]:
top_ids

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0009fFIM1eYThaPg,xP8c9HS5rqvmriRg,aLdUurk83dJm8w7W,0XqU6v83ZMOeh4wc,fyQyjwU2vsgFEl1r,9cF6XHF80kGFAZc1,aHSX00Tar4620YO6,BlyUBjyyZ8wEC4Lg,dMcy1JLT4iEPweUI,ZjM2cKNxK9U3i1xY,cWXpvnMaoOLOcPZx,...,5B1lk8SkefODDlF9,07AjGMk31TkPW3O9,mMNCmXVgaCTVDSuc,7EjNBvMzGWHNeUyN,H06n5EiDyQyxwII6,0WniuVWwli1ez0mR,MHjzLyvCm9q7I5JN,wLCTKkzIN7y6BLoZ,wVRYBhesJywYtx5A,W0RexNIZuGKa3Nl9
0010xmHR6UICBOYT,m6YIrcf0iG25NYtS,e9hFdGfKFyxPuQvq,UD0DWvRMwWe8Swiw,VndT0hzLfaNINtjf,6tIlBkAS3hP8ohzs,rt8pmZEOoGWbt03f,9EDGcoex4PWXVhRs,9w9gcgpKBBcZMXHL,FADoCNlS4nOrax2n,EuVg4Ou0zOYx3Zzd,...,LKyKOXUr3NkTOYNe,xdyzNr7Gisl5eeWV,TcClCCLPQHTLN5BX,phrRAxDxoOmOm2VH,KDBrwVMA0iltXJit,0FBYAHnVYyF7w6Vm,vhVXY7BApvrxuCTs,5HsgZD8nSKlQxrVd,UMnH6qMfqkSmhACq,2WwpcPbpW7WUN85S
002Jyd0vN4HyCpqL,YHupaKUf3ndWCU0v,VllNteG3w18TMKYP,XaWFvftldmxTyE23,a2dMh46H35G46DRb,SB1NTRVKoQiFQUR0,dG79PeWA0lQCVaHu,SxGfnDda6gKj1dxH,ZlcuTUFRxPYCuBwJ,TKTKbwbDl845STJF,b93f0GYGzdKlq4I7,...,2QonsnOn5Tc7Rt5h,QMqyB2OWGCjPOs41,LxaXC6gP3Z9RsD2d,axIWadanvvzn3Pvx,ceRyPWY7XBnvF92A,IyOjhsVVei0LXUw6,XL9ttv6KtHJ4vyhO,lwjCqRX8vexafTOd,W5i36XhNBJv4RiIU,EezQt3u48rZ7scpC
006TYKNjNxWjfKjy,0cYTIIElIl2tFyoW,uMRJJ3QnZ1mGkCQX,6GneFoGYxFqPeH4S,zI3j3T5qkiylIBaD,9eZiNeqg0CQXDRIk,QTxf0P1qd3Dsrc5D,HOC0zk7yRxRPkCTJ,7857AeIHifquoNmK,UceDke4Q33GCS6nY,iA9KEeDlCsgMyQRo,...,Ly4mxnhHOl4QPHBx,gcbSCgxVlxiHvwic,XYivSoFfW0AiMqr3,wR4F3A90kVplf4bJ,7TTPpNjvr4oqjCM4,Y8qPSzqRMxSHEjRR,qqtMvrE3derdpeCk,BUuoaQWXJ1QxJl5o,0sxXq0PI2wB9TryG,0RRt9hQD1qGd8pfz
007LIJOPQ4Sb98qV,qsmfGwfCd4FFaDet,ljHcf8k4kcRQ525P,cyRMqYRBDcRleNsa,9epP1yOXfLKlkR3S,S1rroi8TiRqPNWDX,a2bC8PID2eQzeNFn,0Hhuy13DlIS8loEY,0Sc2Xuj448Am1PkD,wOhO1MgW1CcEKaAx,2z29YJphtXmG8xqV,...,jNQVI254qWEotBEw,H60atNftm3VPVItO,HMvRdVsvXUPwROWw,2qwvTDCDqz7BkpDO,rHxMh2TRcYgoOYrR,bSg4cOjecHcp2Zmt,0VWbBD7x9WKDXitt,9h29AurKytmnDTUS,ldQsK4wsh2IaeGe9,Xzpc3XmT0qJsl3Hf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzx8CWdM7qkxKQpC,pF2UjM0abOPwWfBm,23z7bAcQF1DACqpE,CGz0z5iv0jZmIa7X,9833DTWDgiQRwUqO,UEH28vtjfDBIbDRd,wZWxThEcPCOUnikc,r38azxVlSiKyvYp1,6dV3AoxUu9q2Uwof,1WmILK0emIsdTSl2,9HylO3KnucN7ifb4,...,U5SwPqTnpXjk9bC6,Ha9EdV4sQhxFJgo1,7dqVUMxVHgaOVns4,RmdOD9pdZW2rxWA8,2f72N3aUrcGbVsm6,UDb8MYNDkrMToclA,IBvdSGHX9PGVNKOw,lTuFCTvvBnrDPW5z,Ix3E8QzXaW2l60z0,QyCGEzZqTCrx7IdL
zzyb5LvKJTWLVnrk,D5z0dWO50sLyTDWE,3bejCrhNQlm97r9A,0Pn0EYWzup45hJB5,qaeZAwbRSlV5IB4o,IbdeoyURhiHVZuIX,pF5dqrJwRreBWon6,R860DzF4ox5cXCoz,avSFhKA5GZMDAuLM,ffw6DfI0KN2VIIck,QO41p5UUGWGmiCDr,...,fW8ddcjGyQXRLBp0,qAURhbnXZ0tXwDPx,8Wn5e3q1UnNucOtP,JRrK2n6spoDttKOj,pkzbnslE6Gk7KYlF,zVBVi5hr9JqO5QYg,UCHsDKSTmeSEouRV,RTsAYYQvkPu0YjUU,2549kZXl7oTGSd8j,V4IMQpm3iIKay5RE
zzz0n04uuTUA7fNh,uRNge3sN74NdIaYi,lNdmadPCTL6vlaaQ,HwFgTWD3DyWJnDOm,9EdjT3tfTfcfq2NM,0YxiZtygt6zXSnTr,e42BDZS0sZlTx5mu,Es6aw5ozgNnKbPrF,uHNKT1Meu35bah3W,U42mMUpgGJnphDDh,Pk2dvqU8cnutBoI5,...,ZOyppcq7tVe4WoiS,DPGhN6RAvGYvEVYY,aRG8XY1YZQMi9UCL,7wubDavxrYllvCMx,0SwtGR8MlkcIxCMm,0m1MyuwLx1EjuQqD,iqmbCdX5wUMYhLwI,05ogun9U0TYSNQDO,6eMUgKOa9q4mlow1,4S8s0vwC5BSH2Npt
zzznMjZAKnJJXQSj,m3bU7wEiG8i3QgLU,dfMDtC99XxwJBTho,TTZ6nOkz6a3yO45h,2mzlldXgCYcvYcg7,igPQmB6xcoLu5kvm,GmU3UHKt1NXz19w4,7PCTOXUwR2lO6jQy,okMdFbMjVmEb4JZK,arJV2x5YxfzhWhHT,IOnrB7htO4atg5dt,...,SQ1Lgea7H5jpASWg,OuH9eQhzvWEg0T78,ydLADICszvB61OtR,Pqx3xJee7SI6aSkP,0Lgc4YsFJTKIOTZQ,Tu4wqjgfjYm3DCmM,TIDAYplHydmOIiMl,jwNo0P17Ji0EVyIC,CY2PCIr7j680lCzs,fL7PtttkjSX8s6tP


In [30]:
results = {
    "MAP@10": meanAveragePrecision(top_ids, 10, genres),
    "MAP@100": meanAveragePrecision(top_ids, 100, genres),
    "MRR@10": meanReciprocalRank(top_ids, 10, genres),
    "MRR@100": meanReciprocalRank(top_ids, 100, genres),
    "NDCG@10": ndcgMean(top_ids, 10, genres)[1],
    "NDCG@100": ndcgMean(top_ids, 100, genres)[1],
}

  0%|          | 0/68641 [00:00<?, ?it/s]/DATA/myprograms_Python/MMSR/project/functions.py:111: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:111: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:111: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:111: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:111: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_Python/MMSR/project/functions.py:111: RuntimeWarning: invalid value encountered in double_scalars
  ndcg.append(np.sum(dcg) / np.sum(idcg))
/DATA/myprograms_

In [31]:
results

{'MAP@10': 0.7448601544809241,
 'MAP@100': 0.6557212476718379,
 'MRR@10': 0.8058978991916305,
 'MRR@100': 0.8079706530244403,
 'NDCG@10': nan,
 'NDCG@100': nan}